In [ ]:
%matplotlib inline


# Random balanced network (exp synapses, multiple time constants)

This script simulates an excitatory and an inhibitory population on
the basis of the network used in [1]_.

The example demonstrate the usage of the multisynapse neuron
model. Each spike arriving at the neuron triggers an exponential
PSP. The time constant associated with the PSP is defined in the
receptor type array tau_syn of each neuron. The receptor types of all
connections are uniformally distributed, resulting in uniformally
distributed time constants of the PSPs.

When connecting the network, customary synapse models are used, which
allow for querying the number of created synapses. Using spike
recorders, the average firing rates of the neurons in the populations
are established. The building as well as the simulation time of the
network are recorded.

## References

.. [1] Brunel N (2000). Dynamics of sparsely connected networks of excitatory and
       inhibitory spiking neurons. Journal of Computational Neuroscience 8,
       183-208.

## See Also

:doc:`brunel_alpha_nest`


Import all necessary modules for simulation, analysis and plotting.



In [ ]:
import time
import nest
import nest.raster_plot
import matplotlib.pyplot as plt

nest.ResetKernel()

Assigning the current time to a variable in order to determine the build
time of the network.



In [ ]:
startbuild = time.time()

Assigning the simulation parameters to variables.



In [ ]:
dt = 0.1    # the resolution in ms
simtime = 1000.0  # Simulation time in ms
delay = 1.5    # synaptic delay in ms

Definition of the parameters crucial for asynchronous irregular firing of
the neurons.



In [ ]:
g = 5.0  # ratio inhibitory weight/excitatory weight
eta = 2.0  # external rate relative to threshold rate
epsilon = 0.1  # connection probability

Definition of the number of neurons in the network and the number of neurons
recorded from



In [ ]:
order = 2500
NE = 4 * order  # number of excitatory neurons
NI = 1 * order  # number of inhibitory neurons
N_neurons = NE + NI   # number of neurons in total
N_rec = 50      # record from 50 neurons

Definition of connectivity parameters



In [ ]:
CE = int(epsilon * NE)  # number of excitatory synapses per neuron
CI = int(epsilon * NI)  # number of inhibitory synapses per neuron
C_tot = int(CI + CE)      # total number of synapses per neuron

Initialization of the parameters of the integrate and fire neuron and the
synapses. The parameters of the neuron are stored in a dictionary.



In [ ]:
tauMem = 20.0  # time constant of membrane potential in ms
theta = 20.0  # membrane threshold potential in mV
J = 0.1   # postsynaptic amplitude in mV
nr_ports = 100  # number of receptor types
# Create array of synaptic time constants for each neuron,
# ranging from 0.1 to 1.09 ms.
tau_syn = [0.1 + 0.01 * i for i in range(nr_ports)]
neuron_params = {"C_m": 1.0,
                 "tau_m": tauMem,
                 "t_ref": 2.0,
                 "E_L": 0.0,
                 "V_reset": 0.0,
                 "V_m": 0.0,
                 "V_th": theta,
                 "tau_syn": tau_syn}
J_ex = J       # amplitude of excitatory postsynaptic current
J_in = -g * J_ex  # amplitude of inhibitory postsynaptic current

Definition of threshold rate, which is the external rate needed to fix the
membrane potential around its threshold, the external firing rate and the
rate of the poisson generator which is multiplied by the in-degree CE and
converted to Hz by multiplication by 1000.



In [ ]:
nu_th = theta / (J * CE * tauMem)
nu_ex = eta * nu_th
p_rate = 1000.0 * nu_ex * CE

Configuration of the simulation kernel by the previously defined time
resolution used in the simulation. Setting ``print_time`` to `True` prints the
already processed simulation time as well as its percentage of the total
simulation time.



In [ ]:
nest.resolution = dt
nest.print_time = True
nest.overwrite_files = True
nest.local_num_threads = 4

print("Building network")

Creation of the nodes using ``Create``. We store the returned handles in
variables for later reference. Here the excitatory and inhibitory, as well
as the poisson generator and two spike recorders. The spike recorders will
later be used to record excitatory and inhibitory spikes. Properties of the
nodes are specified via ``params``, which expects a dictionary.



In [ ]:
nodes_ex = nest.Create("iaf_psc_exp_multisynapse", NE, params=neuron_params)
nodes_in = nest.Create("iaf_psc_exp_multisynapse", NI, params=neuron_params)
noise = nest.Create("poisson_generator", params={"rate": p_rate})
espikes = nest.Create("spike_recorder")
ispikes = nest.Create("spike_recorder")

Configuration of the spike recorders recording excitatory and inhibitory
spikes by sending parameter dictionaries to ``set``. Setting the property
`record_to` to *"ascii"* ensures that the spikes will be recorded to a file,
whose name starts with the string assigned to the property `label`.



In [ ]:
espikes.set(label="brunel-py-ex", record_to="ascii")
ispikes.set(label="brunel-py-in", record_to="ascii")

print("Connecting devices")

Definition of a synapse using ``CopyModel``, which expects the model name of
a pre-defined synapse, the name of the customary synapse and an optional
parameter dictionary. The parameters defined in the dictionary will be the
default parameter for the customary synapse. Here we define one synapse for
the excitatory and one for the inhibitory connections giving the
previously defined weights and equal delays.



In [ ]:
nest.CopyModel("static_synapse", "excitatory",
               {"weight": J_ex, "delay": delay})
nest.CopyModel("static_synapse", "inhibitory",
               {"weight": J_in, "delay": delay})

Connecting the previously defined poisson generator to the excitatory and
inhibitory neurons using the excitatory synapse. Since the poisson
generator is connected to all neurons in the population the default rule
(# ``all_to_all``) of ``Connect`` is used. The synaptic properties are
pre-defined # in a dictionary and inserted via ``syn_spec``. As synaptic model
the pre-defined synapses "excitatory" and "inhibitory" are choosen,
thus setting ``weight`` and ``delay``. The receptor type is drawn from a
distribution for each connection, which is specified in the synapse
properties by assigning a dictionary to the keyword ``receptor_type``,
which includes the specification of the distribution and the associated
parameter.



In [ ]:
syn_params_ex = {"synapse_model": "excitatory",
                 "receptor_type": nest.random.uniform_int(max=nr_ports - 1) + 1
                 }
syn_params_in = {"synapse_model": "inhibitory",
                 "receptor_type": nest.random.uniform_int(max=nr_ports - 1) + 1
                 }

nest.Connect(noise, nodes_ex, syn_spec=syn_params_ex)
nest.Connect(noise, nodes_in, syn_spec=syn_params_ex)

Connecting the first ``N_rec`` nodes of the excitatory and inhibitory
population to the associated spike recorders using excitatory synapses.
Here the same shortcut for the specification of the synapse as defined
above is used.



In [ ]:
nest.Connect(nodes_ex[:N_rec], espikes, syn_spec="excitatory")
nest.Connect(nodes_in[:N_rec], ispikes, syn_spec="excitatory")

print("Connecting network")

print("Excitatory connections")

Connecting the excitatory population to all neurons while distribution the
ports. Here we use the previously defined parameter dictionary
``syn_params_ex``. Beforehand, the connection parameter are defined in a
dictionary. Here we use the connection rule ``fixed_indegree``,
which requires the definition of the indegree.



In [ ]:
conn_params_ex = {'rule': 'fixed_indegree', 'indegree': CE}
nest.Connect(nodes_ex, nodes_ex + nodes_in, conn_params_ex, syn_params_ex)

print("Inhibitory connections")

Connecting the inhibitory population to all neurons while distribution the
ports. Here we use the previously defined parameter dictionary
``syn_params_in``.The connection parameter are defined analogously to the
connection from the excitatory population defined above.



In [ ]:
conn_params_in = {'rule': 'fixed_indegree', 'indegree': CI}
nest.Connect(nodes_in, nodes_ex + nodes_in, conn_params_in, syn_params_in)

Storage of the time point after the buildup of the network in a variable.



In [ ]:
endbuild = time.time()

Simulation of the network.



In [ ]:
print("Simulating")

nest.Simulate(simtime)

Storage of the time point after the simulation of the network in a variable.



In [ ]:
endsimulate = time.time()

Reading out the total number of spikes received from the spike recorder
connected to the excitatory population and the inhibitory population.



In [ ]:
events_ex = espikes.n_events
events_in = ispikes.n_events

Calculation of the average firing rate of the excitatory and the inhibitory
neurons by dividing the total number of recorded spikes by the number of
neurons recorded from and the simulation time. The multiplication by 1000.0
converts the unit 1/ms to 1/s=Hz.



In [ ]:
rate_ex = events_ex / simtime * 1000.0 / N_rec
rate_in = events_in / simtime * 1000.0 / N_rec

Reading out the number of connections established using the excitatory and
inhibitory synapse model. The numbers are summed up resulting in the total
number of synapses.



In [ ]:
num_synapses = (nest.GetDefaults("excitatory")["num_connections"]
                + nest.GetDefaults("inhibitory")["num_connections"])

Establishing the time it took to build and simulate the network by taking
the difference of the pre-defined time variables.



In [ ]:
build_time = endbuild - startbuild
sim_time = endsimulate - endbuild

Printing the network properties, firing rates and building times.



In [ ]:
print("Brunel network simulation (Python)")
print(f"Number of neurons : {N_neurons}")
print(f"Number of synapses: {num_synapses}")
print(f"       Exitatory  : {int(CE * N_neurons) + N_neurons}")
print(f"       Inhibitory : {int(CI * N_neurons)}")
print(f"Excitatory rate   : {rate_ex:.2f} Hz")
print(f"Inhibitory rate   : {rate_in:.2f} Hz")
print(f"Building time     : {build_time:.2f} s")
print(f"Simulation time   : {sim_time:.2f} s")

Plot a raster of the excitatory neurons and a histogram.



In [ ]:
nest.raster_plot.from_device(espikes, hist=True)
plt.show()

[![EBRAINS Notebook](https://nest-simulator.org/TryItOnEBRAINS.png)](https://lab.ebrains.eu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fjessica-mitchell%2Fnest-simulator&urlpath=lab%2Ftree%2Fnest-simulator%2Fpynest%2Fexamples%2Fnotebooks%2Fbrunel_exp_multisynapse_nest.ipynb&branch=ebrains-button)
